In [2]:
%pip install pymongo
%pip install python-dotenv
# %pip install requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Importación de librerias

In [3]:
import pymongo
# import requests
# import time
import os
import pandas as pd

# from datetime import datetime
from dotenv import load_dotenv
# from web3 import Web3

c:\Users\lealp\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\lealp\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\lealp\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\lealp\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\lealp\AppData\Local\Temp\ipykernel_13356\2074460846.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,

Carga los secretos del archivo **.env**

In [4]:
# Carga los secretos del archivo .env
load_dotenv()

# Accede a los valores definidos en .env
APIKEY_ETHERSCAN = os.getenv("APIKEY_ETHERSCAN")
MONGODB_SERVER = os.getenv("MONGODB_SERVER")

print(APIKEY_ETHERSCAN[:4]+('*'*10)+APIKEY_ETHERSCAN[-4:])
print(MONGODB_SERVER[:15] + '*****' + MONGODB_SERVER[25:])

1UXP**********R783
mongodb://local*****/


### Conexión MongoDB

In [5]:
mongoClient = pymongo.MongoClient(MONGODB_SERVER, serverSelectionTimeoutMS=5000)

try:
    print(mongoClient.server_info())
except Exception:
    print("Unable to connect to the server.")

{'version': '5.0.3', 'gitVersion': '657fea5a61a74d7a79df7aff8e4bcf0bc742b748', 'targetMinOS': 'Windows 7/Windows Server 2008 R2', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [5, 0, 3, 0], 'openssl': {'running': 'Windows SChannel'}, 'buildEnvironment': {'distmod': 'windows', 'distarch': 'x86_64', 'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.26.28806 for x64', 'ccflags': '/nologo /WX /FImongo/platform/basic.h /EHsc /W3 /wd4068 /wd4244 /wd4267 /wd4290 /wd4351 /wd4355 /wd4373 /wd4800 /wd4251 /wd4291 /we4013 /we4099 /we4930 /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /permissive- /Zc:__cplusplus /Zc:sizedDealloc /volatile:iso /diagnostics:caret /std:c++17 /Gw /Gy /Zc:inline', 'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.26.28806 for x64', 'cxxflags': '/TP', 'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF', 'target_arch': 'x86_64', 'target_os': 'windows', 'cppdefine

Definición BBDD, colecciones e indices en MongoDB

In [6]:
# Base de Datos
mongoDb = mongoClient['web3_tfm']

# Colección Históricos Bloques
collection_block_history = mongoDb['block_history']

# Crear índice único para 'date' e indice para 'max_block_number'
collection_block_history.create_index('date')
collection_block_history.create_index('max_block_number')

# Colección Directorio de Cuentas
collection_account_directory = mongoDb['account_directory']

# Colección Directorio de Redes
collection_network_directory = mongoDb['network_directory']

# Colección Tipo de Cambio Ether / Dollar Americano
collection_eth_usd_quotes = mongoDb['eth_usd_quotes']

# Colección Tipo de Cambio Ether / Euro
collection_eth_eur_quotes = mongoDb['eth_eur_quotes']

### Carga colección *block_history*
Lectura fichero CSV

In [16]:
csv_file = r"data\ethereum-historico-bloques-fin-dia.csv"

# Leemos el archivo CSV
df = pd.read_csv(csv_file)

# Imprimimos la cabecera del DataFrame
print(df.head())

         date  max_block_number
0  2015-07-30              6911
1  2015-07-31             13774
2  2015-08-01             19067
3  2015-08-02             24425
4  2015-08-03             29705


Borra todos los datos en la colección para cargar todo el contenido del fichero CSV

In [71]:
result = collection_block_history.delete_many({})

# Comprueba el número de documentos eliminados
print(f"Número de documentos eliminados: {result.deleted_count}")

Número de documentos eliminados: 0


Preparación de datos
- Añade el identificador de la red a la que pertenecen los bloques (1 - Ethereum)  
- Añade el número del día de la semana para facilitar el filtrado de los datos  

In [72]:
# Se incluye un 3º atributo para identificar la red blockchain de la que se ha obtenido los datos
df["id_network"] = 1
df['day_week'] = pd.to_datetime(df['date']).dt.dayofweek
df.head()

,date,max_block_number,id_network,day_week
0,2015-07-30,6911,1,3
1,2015-07-31,13774,1,4
2,2015-08-01,19067,1,5
3,2015-08-02,24425,1,6
4,2015-08-03,29705,1,0


Carga de datos en MongoDB

In [73]:
# Convertir el DataFrame de Pandas a una lista de diccionarios
data = df.to_dict(orient='records')

# Insertar los datos en la colección MongoDB
result = collection_block_history.insert_many(data)

# Imprime el resultado
print(f"Archivo CSV {csv_file} cargado en MongoDB 'block_history'.")
print(f"{len(result.inserted_ids)} documentos insertados.")

Archivo CSV data\ethereum-historico-bloques-fin-dia.csv cargado en MongoDB 'block_history'.
3123 documentos insertados.


In [74]:
print(data[0])

{'date': '2015-07-30', 'max_block_number': 6911, 'id_network': 1, 'day_week': 3, '_id': ObjectId('65d140b53cd0e5823d7dda18')}


Selecciona unicamente las bloques a partir del año 2023

In [75]:
filtro_fecha = df["date"] > "2022-12-31"
numeros_bloque = df[filtro_fecha]["max_block_number"][:10].tolist()
print(len(numeros_bloque), numeros_bloque)

10 [16315361, 16322532, 16329700, 16336870, 16344023, 16351183, 16358346, 16365511, 16372678, 16379844]


**Función para la carga de colecciones a partir de parámetros en ficheros CSV**
- Lee ficheros CSV
- Borra todos los elementos en la colección
- Inserta datos en fichero CSV

In [11]:
def load_collection_from_data(collection, dataframe):
    
    # Convertir el DataFrame de Pandas a una lista de diccionarios
    data = dataframe.to_dict(orient='records')
    
    # Borrado de datos para cargar todo el contenido del fichero CSV
    result = collection.delete_many({})

    # Comprueba el número de documentos eliminados
    print(f"\nNúmero de documentos eliminados: {result.deleted_count}")
    
    # Insertar los datos en la colección MongoDB
    result = collection.insert_many(data)

    # Imprime el resultado
    print(f"\nArchivo CSV {csv_file} cargado en base de datos. Documentos insertados: {len(result.inserted_ids)}")
    print("Elemento insertado:")
    print(data[0])

In [12]:
def load_collection(collection, csv_file):
    
    # Leer el archivo CSV con Pandas
    df = pd.read_csv(csv_file, delimiter=';')
    print(df.head())
    
    load_collection_from_data(collection, df)
    

### Carga colección *account_directory*

In [20]:
# Ruta archivo CSV
csv_file = r'data/account_directory.csv'

# Carga de datos
load_collection(collection_account_directory, csv_file)

   from_network  to_network  layer  \
0             1       42161      1   
1             1       42161      1   
2             1       42161      1   
3             1       42161      1   
4             1       42161      1   

                                      address description  \
0  0x4Dbd4fc535Ac27206064B68FfCf827b0A60BAB3f    Arbitrum   
1  0xa3A7B6F88361F48403514059F1F16C8E78d60EeC    Arbitrum   
2  0xcEe284F754E854890e311e3280b767F80797180d    Arbitrum   
3  0xd92023E9d9911199a6711321D1277285e6d4e2db    Arbitrum   
4  0xA10c7CE4b876998858b1a9E12b10092229539400    Arbitrum   

                                    full_description  is_transactions  \
0  Arbitrum Official Bridge Contract - Delayed Inbox             True   
1                     Arbitrum One: L1 ERC20 Gateway            False   
2              Arbitrum One: L1 Arb - Custom Gateway            False   
3                Arbitrum One: Wrapped Ether Gateway            False   
4                            Arbitrum: 

### Carga colección *network_directory*

In [78]:
# Ruta archivo CSV
csv_file = r'data/network_directory.csv'

# Carga de datos
load_collection(collection_network_directory, csv_file)

      id  id_hex          value       description  isMainnet
0      1     0x1       ethereum          Ethereum       True
1  42161  0xa4b1       arbitrum      Arbitrum One       True
2  42170  0xa4ba  arbitrum_nova     Arbitrum Nova       True
3     10     0xa       optimism  Optimism Mainnet       True
4    137    0x89        polygon   Polygon Mainnet       True

Número de documentos eliminados: 0

Archivo CSV data/network_directory.csv cargado en base de datos. Documentos insertados: 96
Elemento insertado:
{'id': 1, 'id_hex': '0x1', 'value': 'ethereum', 'description': 'Ethereum', 'isMainnet': True, '_id': ObjectId('65d140ce3cd0e5823d7de656')}


### Carga colección *eth_usd_quotes*

In [24]:
# Ruta archivo CSV
csv_file = r'data/eth-usd-max.csv'

# Carga de datos
load_collection(collection_eth_usd_quotes, csv_file)

                snapped_at        price   market_cap total_volume
0  2015-08-07 00:00:00 UTC      2,83162            0        90622
1  2015-08-08 00:00:00 UTC      1,33075     80339475       368070
2  2015-08-10 00:00:00 UTC  0,687585709  41556309,21  400464,0745
3  2015-08-11 00:00:00 UTC  1,067378879  64539006,31  1518998,205
4  2015-08-12 00:00:00 UTC  1,256613175  76013261,05  2073893,345

Número de documentos eliminados: 0

Archivo CSV data/eth-usd-max.csv cargado en base de datos. Documentos insertados: 3119
Elemento insertado:
{'snapped_at': '2015-08-07 00:00:00 UTC', 'price': '2,83162', 'market_cap': '0', 'total_volume': '90622', '_id': ObjectId('65d5a5733ebb3c4bb440e41c')}


### Carga colección *eth_eur_quotes*

In [18]:
# Ruta archivo CSV
csv_file = r'data/Histórico-ether-euro-investing.csv'

# Leer el archivo CSV en un DataFrame de Pandas
df = pd.read_csv(csv_file)

# Convertir el primer campo a formato de fecha
df['date'] = pd.to_datetime(df['Fecha'], format='%d.%m.%Y')

# Eliminar el punto y reemplazar la coma por punto como separador decimal en los campos 2, 3, 4 y 5
for col in df.columns[1:5]:
    df[col] = df[col].str.replace('.', '').str.replace(',', '.').astype(float)

# Reemplazar la coma por punto como separador decimal en el campo "Vol."
df['volume'] = df['Vol.'].str.replace(',', '.')

# Reemplazar la coma por punto como separador decimal en el campo "% var."
df['variation %'] = df['% var.'].str.replace(',', '.').str.rstrip('%').astype(float)

print(df.head())

load_collection_from_data(collection_eth_eur_quotes, df)

       Fecha   Último  Apertura   Máximo   Mínimo     Vol.  % var.
0 2024-02-20  3015.38   2944.79  3030.61  2879.82  674.55K    2.40
1 2024-02-19  2944.80   2880.16  2983.48  2859.82  478.68K    2.25
2 2024-02-18  2880.10   2785.90  2892.79  2765.97  391.11K    3.38
3 2024-02-17  2785.90   2801.80  2804.17  2721.21  251.97K   -0.57
4 2024-02-16  2801.80   2822.59  2856.76  2759.52  401.19K   -0.74

Número de documentos eliminados: 416

Archivo CSV data/Histórico-ether-euro-investing.csv cargado en base de datos. Documentos insertados: 416
Elemento insertado:
{'Fecha': Timestamp('2024-02-20 00:00:00'), 'Último': 3015.38, 'Apertura': 2944.79, 'Máximo': 3030.61, 'Mínimo': 2879.82, 'Vol.': '674.55K', '% var.': 2.4, '_id': ObjectId('65dbdf756e580e93a40f14c1')}
